In [ ]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import time

import numpy as np
import jax
import jax.numpy as jnp
import jax.lax as lax
from jaxtyping import Array, Float, Int, PRNGKeyArray
import aeon

from features.sig_trp import SigVanillaTensorizedRandProj, SigRBFTensorizedRandProj
from features.sig import SigTransform, LogSigTransform
from features.base import TimeseriesFeatureTransformer, TabularTimeseriesFeatures, RandomGuesser
from features.sig_neural import RandomizedSignature
from utils import print_name, print_shape

from preprocessing.timeseries_augmentation import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from aeon.classification.sklearn import RotationForestClassifier
from sklearn.metrics import accuracy_score

jax.config.update('jax_platform_name', 'gpu') # Used to set the platform (cpu, gpu, etc.)
np.set_printoptions(precision=3, threshold=5) # Print options

from aeon.datasets.tsc_datasets import multivariate, univariate, univariate_equal_length
from aeon.datasets import load_classification
import pandas as pd

# Univariate

In [ ]:
from run_univariate import do_experiments

#run experiments
d_res, d_meta, d_failed = do_experiments(["Adiac", "GunPoint"])

# make dict of results
model_names = d_res[list(d_res.keys())[0]][0][0]
alpha_names = ["alpha_" + model_name for model_name in model_names]
df_accs = pd.DataFrame({dataset : accs for dataset, ((model_names, accs, alphas),) in d_res.items()}).transpose()
df_accs.columns = model_names
df_alphas = pd.DataFrame({dataset : alphas for dataset, ((model_names, accs, alphas),) in d_res.items()}).transpose()
df_alphas.columns = alpha_names
meta = pd.DataFrame(d_meta).transpose()
failed = pd.DataFrame(d_failed).transpose()

df_accs = pd.concat([meta, df_accs], axis=1)
df_alphas = pd.concat([meta, df_alphas], axis=1)

# save
df_accs.to_pickle("df_accs_univariateTSC.pkl")
df_accs.to_pickle("df_alphas_univariateTSC.pkl")
df_accs.to_pickle("df_failed_univariateTSC.pkl")